<pre>














</pre>

# Quick Python And Guaging Interests via Twitter With Sawal And Abhishek
<pre>














</pre>

In [35]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
from twitter import *
import json 
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')
from  geopy.geocoders import Nominatim

<pre>














</pre>

### We will now scroll fast so you don't note down our access tokens, as sir said. Hehe, please don't.
<pre>














</pre>

In [2]:
# keys and tokens from the Twitter Dev Console 
consumer_key = 'Z80SYZFDadQ9JdWT8NgG3964g'
consumer_secret = 'dP7R0Zd24YvEvnDUxq8NuecpF6sawPdJEwmNp8BuvpkDHRu976'
access_token = '1246065714793820160-aXNNEPX87TbVCjBE9MnEFAXi9LERRQ'
access_token_secret = 'c1zmGHmsTNGkbCIIXop7yfLWkoo0i3tSdU1KjhUF7RhMj'

<pre>














</pre>

# Safe Zone Now.
<pre>














</pre>

In [36]:
auth = OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=False)

<pre>




</pre>
### We start by creating some functions that help us throughout the way. As we were always told to not repeat ourselves, we have tried to break down most of our code into functions that can be separately called as per our ease anywhere in the notebook. It is a good way to review on what we learned, which was:

# DON'T REPEAT YOURSELF.
<pre>





</pre>

<pre>


</pre>

# This function cleans the text it receives. We'll use it to clean tweets and get rid of emoticons and special letters. And it will return us the cleaned tweet.
<pre>


</pre>

In [37]:
def clean_tweet(tweet): 
    ''' 
    Utility function to clean tweet text by removing links, special characters 
    using simple regex statements. 
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

<pre>


</pre>

### We initially found sentiment analysis too tough, but then as time went on, we felt that since tweets are such a personal business, no matter how big a dictionary we make of words handpicked ourselves, it won't be enough to guage everything that the users are trying to convey.

## So we used the standard module TextBlob to do sentiment analysis of the tweets we obtain.
<pre>


</pre>
## Below is a function which does so to any text passed to it. The module takes the text and assigns to it a rating or a polarity which can be easily categorized as Positive, Negative or Neutral.
<pre>


</pre>

In [38]:
def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

<pre>


</pre>
# We then moved to creating a function that could analyse multiple tweets from a list of tweets obtained. And after analyzing all of them, it could sum up the numbers and give us percentages of the positivity and the negativity.
<pre>


</pre>
### Below is the function that receives a dataframe of the tweets that we search and obtain with a column 'text' containing all the tweet texts and it also receives 'country' variable which brings with it any location we want to focus on. 

In [39]:
def get_sentiment(df,country,graph=True,show=False):
    tweets = []
    fetched_tweets = df.text
    len_tweets=1
    for tweet in fetched_tweets: 
            # empty dictionary to store required params of a tweet 
            parsed_tweet = {} 

            # saving text of tweet 
            parsed_tweet['text'] = tweet 
            # saving sentiment of tweet 
            parsed_tweet['sentiment'] = get_tweet_sentiment(tweet) 

            tweets.append(parsed_tweet) 
  
    if len(tweets)==0:
        len_tweets = 1
    else: 
        len_tweets = len(tweets)
        
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    posp = round(100*len(ptweets)/len_tweets,2)
    print("Positive tweets percentage: {} %".format(posp)) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    negp = round(100*len(ntweets)/len_tweets,2)
    print("Negative tweets percentage: {} %".format(negp)) 
    # percentage of neutral tweets 
    neutp = round(100*(len(tweets) - len(ntweets) - len(ptweets))/len_tweets,2)
    print("Neutral tweets percentage: {} %".format(neutp))  
    
    if show==True:
        # printing first 5 positive tweets 
        print("\n\nPositive tweets:") 
        count = 0
        for tweet in ptweets[:10]: 
            count+=1
            print(f'{count}. '+'\033[1m'+tweet['text']+'\033[0m') 
        count = 0
        # printing first 5 negative tweets 
        print("\n\nNegative tweets:") 
        for tweet in ntweets[:10]: 
            count+=1
            print(f'{count}. '+tweet['text']) 
        count = 0
    
    
    if graph==True:
        country_result_dict = {'Positive':[posp], 'Negative':[negp], 'Neutral':[neutp]}
        df1 = pd.DataFrame(country_result_dict)
        df1['Country'] = country
        return df1

<pre>


</pre>
## When you write code so extensive, you can't manually do anything. So to get location specific tweets, we needed an automated way to get the geo-codes(longitudes and latitudes) as well. 
<pre>


</pre>
## So we developed the below function that takes on any list of locations(country level to district level to your room level) and gets us the geo-codes for all the locations in the list and returns the output as a dataframe as well. 
<pre>


</pre>

In [40]:
def get_coordinates(countrylist):
    lat = []
    long = []
    geolocator = Nominatim()
    for country in countrylist:
        loc = geolocator.geocode(country)
        lat.append(loc.latitude)
        long.append(loc.longitude)
    coord_dict = {'country':countrylist,'lat':lat,'long':long}
    coordinates = pd.DataFrame(coord_dict).set_index('country')
    return coordinates

<pre>


</pre>
# After obtaining all the analysis of the tweets from the distinct locations by our knowledge of modules, functions and dataframes, the next step for us had to be to show off our interactive plotting skills from A4 so that we can get a quick grasp over outlooks of our queries based on the locations they're searched in.
<pre>


</pre>
## And so below is our function that plots any dataframe of sentiments passed into it.

In [41]:
def graph_query(df,query):

    pos = go.Bar(
                x = df.index,
                y = df.Positive,
                name = 'Positive',
                marker = dict(color = 'greenyellow',
                              line=dict(color='white',width=2))

    )

    neg = go.Bar(
                x = df.index,
                y = df.Negative,
                name = 'Negative',
                marker = dict(color = 'crimson',
                              line=dict(color='white',width=2))
    )

    neut = go.Bar(
                x = df.index,
                y = df.Neutral,
                name = 'Neutral',
                marker = dict(color = 'wheat',
                              line=dict(color='white',width=2))
    )

    data = [pos,neg,neut]
    layout = go.Layout(barmode='relative')
    fig1 = go.Figure(data=data, layout = layout)
    fig1.update_layout(
                title = go.layout.Title(
                                        text = f'Location Wise Outlook Of {query}',
                                        xref = 'container',
                                        x = .5
                                        ),
                xaxis = go.layout.XAxis(
                                        title = go.layout.xaxis.Title(
                                                standoff = 0,
                                                text = 'Location',
                                                font = dict(color = 'seagreen', family = 'helvetica', size = 20)
                                                )
                                        ),
                yaxis = go.layout.YAxis(
                                        title = go.layout.yaxis.Title(
                                                standoff = 0,
                                                text = 'Sentiment Percentage',
                                                font = dict(color = 'seagreen', family = 'helvetica', size = 20)
                                                )
                                        )
    )

    fig1.show()
    #khaki ivory greenyellow

<pre>


</pre>
# Now comes the most important function which is our ultimate call.
# Our analyze function takes a query to search, some locations to search it in, the coordinates of those locations.
# It also asks you gently if you'd like a graph for the results, assuming you'll always want a good-looking graph and also the area you want to search in and if you want the tweets from each place shown or not.
<pre>


</pre>

In [42]:
def analyze(query,countries,latlong,graph=True,radius=1000,show=False):
    print(f'\nOutlook of {query} by location:')
    country_wise_df = pd.DataFrame()

    for country in countries:
        print('\n'+country+'\n')

        #Getting tweets by location for the entered query in only English language for sentiment analysis.
        tweets = api.search(q=query, count = 500, lang='en',
                            geocode=str(latlong.loc[country,'lat'])+','+str(latlong.loc[country,'long'])+","+str(radius)+"km")

        #Creating a dict of minimum information from tweets.
        tweet_dict = {'id':[],'text':[],'created_at':[]}
        for tweet in tweets:
            tweet_dict["id"].append(tweet.id) 
            tweet_dict["text"].append(tweet.text) 
            tweet_dict["created_at"].append(tweet.created_at)

        #Creating dataframe from the dict for each country iteration of the for loop.
        df = pd.DataFrame(tweet_dict)
        country_df = get_sentiment(df,country,graph=graph,show=show)

        #Adding each loop df into an overall df which was initialised empty at the beginning.
        country_wise_df = pd.concat((country_wise_df,country_df))
    
    #Preparing the df for graph if graphs are wanted.
    if graph==True:
        country_wise_df = country_wise_df.set_index('Country')

    #Calling the graph generating function if necessary.
    
    if graph==True:
        graph_query(country_wise_df,query)

<pre>






</pre>

# Let's put this analyze function to test, at the risk of it not working out live in this session.
<pre>







</pre>

In [44]:
query = input("Enter anything to search: ") 
location = []

location.append(input("Please enter a location: "))
more = input("More? ")
while not (more=='no' or more=='n'):

    location.append(input("Enter "))
    more =  input("More? ")
    
radius = input('Enter radius to search 10 or 10000: ')
if input('Do you want to see the tweets as well: y/n? ')==('y'):
    show = True 
#location = ['England','USA','New York','France','Spain']
latlong = get_coordinates(location)
analyze(query,location,latlong,radius=radius,graph=True,show=show)

Enter anything to search: Kush Khurana
Please enter a location: Sonipat
More? y
Enter Hisar
More? y
Enter Delhi
More? y
Enter Indore
More? n
Enter radius to search 10 or 10000: 50
Do you want to see the tweets as well: y/n? y

Outlook of Kush Khurana by location:

Sonipat

Positive tweets percentage: 100.0 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 0.0 %


Positive tweets:
1. Professor Kush Khurana was the best thing to happen to me in 2020.


Negative tweets:

Hisar

Positive tweets percentage: 100.0 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 0.0 %


Positive tweets:
1. Thank You Kush Khurana sir. It was wonderful having you this semester.


Negative tweets:

Delhi

Positive tweets percentage: 100.0 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 0.0 %


Positive tweets:
1. Kush Khurana sir, it was very enlightening to learn python from you this semester. We hope you come back next year… https://t.co/Iiabty3JqJ


Negative tweets:


<pre>







</pre>
# PHEW!
<pre>







</pre>



# To demonstrate how local we can take this, we show the discussions about a local political leader across districts in Rajasthan. This can have election and democratic applications as well.
<pre>



</pre>

In [45]:
countries = ['Ajmer','Bikaner','Jaipur','Jaisalmer']
latlong = get_coordinates(countries)
query = 'Sachin Pilot'
analyze(query,countries,latlong,radius=100,graph=True)


Outlook of Sachin Pilot by location:

Ajmer

Positive tweets percentage: 23.81 %
Negative tweets percentage: 19.05 %
Neutral tweets percentage: 57.14 %

Bikaner

Positive tweets percentage: 100.0 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 0.0 %

Jaipur

Positive tweets percentage: 37.5 %
Negative tweets percentage: 18.75 %
Neutral tweets percentage: 43.75 %

Jaisalmer

Positive tweets percentage: 0.0 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 0.0 %


<pre>



</pre>

# But we can't always be entering so many values. So below we prepared a result which will be discussed separately as well from our document.
# Here we will prepare a list of locations corresponding to each person of interest.
<pre>



</pre>
# And then we list it out as a dataframe for ease of reading and handling.

In [46]:
billc = ['USA','KENYA', 'INDIA','UK','JAPAN','BRASIL', 'TURKEY']
muskc = ['USA', 'INDIA', 'FRANCE', 'JAPAN']
mbsc = ['USA', 'UK', 'GERMANY','UAE','IRAN','INDIA']
modic = ['USA','INDIA','UK','ISRAEL','BRASIL']
merkelc = ['FRANCE','RUSSIA','TURKEY']
buffetc = ['USA','UK','JAPAN','KENYA']
trudeauc = ['USA','INDIA','UK','UAE']

person_country_dict = {'person':['Bill Gates', 'Elon Musk', 'MBS','Modi','Merkel','Buffett','Trudeau'], 
                 'country': [billc, muskc, mbsc, modic, merkelc, buffetc, trudeauc]}
person_country = pd.DataFrame(person_country_dict).set_index('person')
person_country

,country
person,
Bill Gates,"[USA, KENYA, INDIA, UK, JAPAN, BRASIL, TURKEY]"
Elon Musk,"[USA, INDIA, FRANCE, JAPAN]"
MBS,"[USA, UK, GERMANY, UAE, IRAN, INDIA]"
Modi,"[USA, INDIA, UK, ISRAEL, BRASIL]"
Merkel,"[FRANCE, RUSSIA, TURKEY]"
Buffett,"[USA, UK, JAPAN, KENYA]"
Trudeau,"[USA, INDIA, UK, UAE]"


<pre>



</pre>

# We then defined a more automated function than the previous analyze function, which only requires us to feed it the query to search.
<pre>



</pre>

In [47]:
def analyze_query(query,graph=True,geotimeouterror=True):
    
    
    #Getting twitter authorization.
    #auth = OAuthHandler(consumer_key, consumer_secret) 
    #auth.set_access_token(access_token, access_token_secret)
    #api = tweepy.API(auth, wait_on_rate_limit=False)

    #Contingency if entered query not in mentioned list of persons.
    if query in person_country.index:        
        countries = person_country.loc[query,'country']
    else:
        countries = person.country.loc['Bill Gates', 'country']

        
        
    #Contingency if function to get coordinates starts to give timeout error temporarily.
    if geotimeouterror==True:    
        latlong = pd.read_csv('/Users/abhishek/Google Drive/Ashoka/Semester 2/4. Computer Programming/ECO6800/Project/longlat.csv').set_index('country')
    else:
        latlong = get_coordinates(countries)
   
    #Calling the analyze function with this automated information.    
    analyze(query,countries,latlong,graph = graph)

<pre>



</pre>

# The geo-timeout-error variable is a boolean being used to flag if the application in the geopy module is facing a timeout error or not. These errors come up randomly through the internet with no fault in the code.

# However for this live presentation, we didn't want to risk it for these persons of interest, and have prepared a csv file containing the coordinates we want them searched in, that is being used as a dataframe from the above cell's if-else condition.
<pre>



</pre>
# So, you see below that the error bool is turned True for this demo cell only.

### You can enter the name of the person you want to search from the list.

In [48]:
geotimeouterror = True  
#Needs to be set to True if a timeout error comes to get coordinates from a local file like: 
#analyze(query,graph,geotimeouterror=True)

query = input("Enter the person to search: Bill Gates/ Elon Musk/ MBS/ Modi/ Merkel/ Buffett/ Trudeau") 
graphinput = input("Do you want graphs: y/n? ")
graph = False
if graphinput in ['y','YES','yes','Yes']:
    graph = True 

analyze_query(query, graph = graph, geotimeouterror = geotimeouterror)

Enter the person to search: Bill Gates/ Elon Musk/ MBS/ Modi/ Merkel/ Buffett/ TrudeauModi
Do you want graphs: y/n? y

Outlook of Modi by location:

USA

Positive tweets percentage: 26.53 %
Negative tweets percentage: 30.61 %
Neutral tweets percentage: 42.86 %

INDIA

Positive tweets percentage: 38.0 %
Negative tweets percentage: 14.0 %
Neutral tweets percentage: 48.0 %

UK

Positive tweets percentage: 34.0 %
Negative tweets percentage: 6.0 %
Neutral tweets percentage: 60.0 %

ISRAEL

Positive tweets percentage: 37.0 %
Negative tweets percentage: 30.0 %
Neutral tweets percentage: 33.0 %

BRASIL

Positive tweets percentage: 29.17 %
Negative tweets percentage: 29.17 %
Neutral tweets percentage: 41.67 %


<pre>



</pre>

# There comes a time when you can be really lazy. And you just want to observe. We've got you covered. We took on everything in the twitter api to find for you the latest hashtags and ran searches on those hashtags as our search queries to see what the people of the world are saying about the things they've trended themselves.
<pre>



</pre>

In [49]:
def hashtags_query(query = 'Obama'):   # Takes a default query for Obama
    # creating object of TwitterClient Class 
    #api = TwitterClient() 
    # calling function to get tweets 
    tweets = api.search(q=query, count = 500, lang='en')

    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if get_tweet_sentiment(tweet.text) == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(round(100*len(ptweets)/len(tweets),2))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if get_tweet_sentiment(tweet.text) == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(round(100*len(ntweets)/len(tweets),2))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} %".format(round(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets),2)))  

    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    count = 0
    for tweet in ptweets[:1]: 
        count+=1
        print(f'{count}. '+tweet.text) 
    count = 0
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:1]: 
        count+=1
        print(f'{count}. '+tweet.text) 
    count = 0


<pre>



</pre>

# Below we have a pre-loaded database that gives us the where-on-earth-id, ie, woeid, on which websites like Twitter, Yahoo Weather, Flickr , etc. operate. 
# So the user only needs to enter the name of the country, the code does the REST while you REST.
<pre>



</pre>

In [50]:
df = pd.read_csv('/Users/abhishek/Google Drive/Ashoka/Semester 2/4. Computer Programming/ECO6800/Misc./woeid.csv')
df.loc[0,'country']='Worldwide'
df = df.groupby('country').head(1).set_index('country')
df.head()
#Replace with the path of the woeid csv file on your computer.

,name,placeType__code,placeType__name,url,parentid,woeid,countryCode
country,,,,,,,
Worldwide,Worldwide,19,Supername,http://where.yahooapis.com/v1/place/1,0,1,NaN
Canada,Winnipeg,7,Town,http://where.yahooapis.com/v1/place/2972,23424775,2972,CA
United Kingdom,Birmingham,7,Town,http://where.yahooapis.com/v1/place/12723,23424975,12723,GB
Dominican Republic,Santo Domingo,7,Town,http://where.yahooapis.com/v1/place/76456,23424800,76456,DO
Guatemala,Guatemala City,7,Town,http://where.yahooapis.com/v1/place/83123,23424834,83123,GT


### Final hashtag analysis call:

In [51]:
twitter = Twitter(auth = OAuth(access_token,
                  access_token_secret,
                  consumer_key,
                  consumer_secret))

country= input("Enter the country to search: ") 
#country_id = client.fetch_woeid(country)
country_id = df.loc[country,'parentid']
results = twitter.trends.place(_id = country_id)
print(f"{country} Trends")
for location in results:
    for trend in location["trends"][:3]:
        print("\n\n ------------------------------------------------\n- \033[1m %s" % trend["name"] + '\033[0m')    
        hashtags_query(trend['name'])

Enter the country to search: India
India Trends


 ------------------------------------------------
-  #MahaRanaPratapJayanti
Positive tweets percentage: 60.0 %
Negative tweets percentage: 3.0 %
Neutral tweets percentage: 37.0 %


Positive tweets:
1. RT @AQUIBMIR7: Tributes to a great  great Maharana Pratap Singh on his birth anniversary.The legendary tales of his sacrifice, valour &amp; pat…


Negative tweets:
1. RT @HUFToday: Ram Singh Tanvar of Gwalior with all his sons, the descendants of Jaimal and Patta, an Afghan contingent led by Hakim Khan Su…


 ------------------------------------------------
-  #PMCaresFund_का_हिसाब_दो
Positive tweets percentage: 28.0 %
Negative tweets percentage: 28.0 %
Neutral tweets percentage: 44.0 %


Positive tweets:
1. For full video click- https://t.co/3TqlWSa6qS
#PMCaresFund_का_हिसाब_दो https://t.co/nNgFbML5x9


Negative tweets:
1. RT @VinayDokania: Finally when @narendramodi spent the money from #PMCaresFund to save Indians from #COVID19 ...

#PMC

<pre>









</pre>

# In the end, to make our work as general as possible, through the modular approach, we have ended with a way of running a search on anything and not just persons, in any place, in any number of places and for any vicinity.
<pre>








</pre>

In [52]:
#Examples

#WHO in Africa
#Red Cross in Israel
#Venezuela Coup in Europe and USA
#Ban China in Japan, South Korea, Spain, UK, France

In [53]:
query = input("Enter anything to search: ") 
location = []

location.append(input("Please enter a location: "))
more = input("More? ")
while not (more=='no' or more=='n'):

    location.append(input("Enter "))
    more =  input("More? ")
    
radius = input('Enter radius to search 100 or 10000: ')
#location = ['England','USA','New York','France','Spain']
latlong = get_coordinates(location)
analyze(query,location,latlong,radius=radius,graph=True)

Enter anything to search: Ban China
Please enter a location: Japan
More? y
Enter South Korea
More? y
Enter Spain
More? y
Enter UK
More? y
Enter France
More? n
Enter radius to search 100 or 10000: 100000

Outlook of Ban China by location:

Japan

Positive tweets percentage: 23.53 %
Negative tweets percentage: 52.94 %
Neutral tweets percentage: 23.53 %

South Korea

Positive tweets percentage: 33.33 %
Negative tweets percentage: 11.9 %
Neutral tweets percentage: 54.76 %

Spain

Positive tweets percentage: 7.5 %
Negative tweets percentage: 87.5 %
Neutral tweets percentage: 5.0 %

UK

Positive tweets percentage: 37.5 %
Negative tweets percentage: 16.67 %
Neutral tweets percentage: 45.83 %

France

Positive tweets percentage: 27.59 %
Negative tweets percentage: 17.24 %
Neutral tweets percentage: 55.17 %


<pre>



</pre>

# And if you want to see the results that Sawal is going to discuss for each of these persons of interest, we have prepared an absolute ram and browser and twitter killer function below:
<pre>



</pre>

In [34]:
def RAMdestroyer():
    for person in person_country.index:
        analyze_query(person)

In [332]:
RAMdestroyer()


Outlook of Bill Gates by location:

USA

Positive tweets percentage: 4.0 %
Negative tweets percentage: 2.0 %
Neutral tweets percentage: 94.0 %

KENYA

Positive tweets percentage: 1.0 %
Negative tweets percentage: 96.0 %
Neutral tweets percentage: 3.0 %

INDIA

Positive tweets percentage: 33.33 %
Negative tweets percentage: 18.52 %
Neutral tweets percentage: 48.15 %

UK

Positive tweets percentage: 40.0 %
Negative tweets percentage: 9.0 %
Neutral tweets percentage: 51.0 %

JAPAN

Positive tweets percentage: 26.0 %
Negative tweets percentage: 24.0 %
Neutral tweets percentage: 50.0 %

BRASIL

Positive tweets percentage: 30.0 %
Negative tweets percentage: 14.0 %
Neutral tweets percentage: 56.0 %

TURKEY

Positive tweets percentage: 39.0 %
Negative tweets percentage: 23.0 %
Neutral tweets percentage: 38.0 %



Outlook of Elon Musk by location:

USA

Positive tweets percentage: 10.0 %
Negative tweets percentage: 12.0 %
Neutral tweets percentage: 78.0 %

INDIA

Positive tweets percentage: 42.0 %
Negative tweets percentage: 4.0 %
Neutral tweets percentage: 54.0 %

FRANCE

Positive tweets percentage: 12.0 %
Negative tweets percentage: 4.0 %
Neutral tweets percentage: 84.0 %

JAPAN

Positive tweets percentage: 21.0 %
Negative tweets percentage: 59.0 %
Neutral tweets percentage: 20.0 %



Outlook of MBS by location:

USA

Positive tweets percentage: 28.75 %
Negative tweets percentage: 21.25 %
Neutral tweets percentage: 50.0 %

UK

Positive tweets percentage: 44.0 %
Negative tweets percentage: 13.0 %
Neutral tweets percentage: 43.0 %

GERMANY

Positive tweets percentage: 21.0 %
Negative tweets percentage: 12.0 %
Neutral tweets percentage: 67.0 %

UAE

Positive tweets percentage: 9.09 %
Negative tweets percentage: 5.45 %
Neutral tweets percentage: 85.45 %

IRAN

Positive tweets percentage: 21.0 %
Negative tweets percentage: 6.0 %
Neutral tweets percentage: 73.0 %

INDIA

Positive tweets percentage: 49.0 %
Negative tweets percentage: 16.0 %
Neutral tweets percentage: 35.0 %



Outlook of Modi by location:

USA

Positive tweets percentage: 45.83 %
Negative tweets percentage: 15.62 %
Neutral tweets percentage: 38.54 %

INDIA

Positive tweets percentage: 29.0 %
Negative tweets percentage: 10.0 %
Neutral tweets percentage: 61.0 %

UK

Positive tweets percentage: 79.0 %
Negative tweets percentage: 6.0 %
Neutral tweets percentage: 15.0 %

ISRAEL

Positive tweets percentage: 37.0 %
Negative tweets percentage: 38.0 %
Neutral tweets percentage: 25.0 %

BRASIL

Positive tweets percentage: 28.38 %
Negative tweets percentage: 28.38 %
Neutral tweets percentage: 43.24 %



Outlook of Merkel by location:

FRANCE

Positive tweets percentage: 38.0 %
Negative tweets percentage: 11.0 %
Neutral tweets percentage: 51.0 %

RUSSIA

Positive tweets percentage: 50.0 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 50.0 %

TURKEY

Positive tweets percentage: 25.37 %
Negative tweets percentage: 11.94 %
Neutral tweets percentage: 62.69 %



Outlook of Buffett by location:

USA

Positive tweets percentage: 31.52 %
Negative tweets percentage: 16.3 %
Neutral tweets percentage: 52.17 %

UK

Positive tweets percentage: 36.0 %
Negative tweets percentage: 37.0 %
Neutral tweets percentage: 27.0 %

JAPAN

Positive tweets percentage: 34.0 %
Negative tweets percentage: 19.0 %
Neutral tweets percentage: 47.0 %

KENYA

Positive tweets percentage: 21.74 %
Negative tweets percentage: 14.13 %
Neutral tweets percentage: 64.13 %



Outlook of Trudeau by location:

USA

Positive tweets percentage: 34.0 %
Negative tweets percentage: 11.0 %
Neutral tweets percentage: 55.0 %

INDIA

Positive tweets percentage: 15.0 %
Negative tweets percentage: 3.0 %
Neutral tweets percentage: 82.0 %

UK

Positive tweets percentage: 48.0 %
Negative tweets percentage: 29.0 %
Neutral tweets percentage: 23.0 %

UAE

Positive tweets percentage: 36.0 %
Negative tweets percentage: 16.0 %
Neutral tweets percentage: 48.0 %
